输入输出通常可以划分为几个大类：读取文本文件和其他更高效的磁盘存储格式，加载数据库中的数据，利用`Web API`操作网络资源。

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
import matplotlib as plt

## 读写文本格式的数据
`pandas`提供了一些用于将表格型数据读取为`DataFrame`对象的I函数。下表对它们进行了总结，其中`read_csv`和`read_table`可能会是你今后用得最多的。  
![](./pandas中的解析函数.png)

我将大致介绍一下这些函数在将文本数据转换为`DataFrame`时所用到的一些技术。这些函数的选项可以划分为以下几个大类：  
•	索引：将一个或多个列当做返回的`DataFrame`处理，以及是否从文件、用户获取列名。  
•	类型推断和数据转换：包括用户定义值的转换、缺失值标记列表等。  
•	日期解析： 包括组合功能，比如将分散在多个列中的日期时间信息组合成结果中的单个列。  
•	迭代：支持对大文件进行逐块迭代。  
•	不规整数据问题： 跳过一些行、页脚、注释或其他一些不重要的东西（比如由成千上万个逗号隔开的数值数据）。  
类型推断(type inference)是这些函数中最重要的功能之一，也就是说,你不需要指定列的类型到底是数值、整数、布尔值，还是字符串。日期和其他自定义类型的处理需要多花点工夫才行。首先我们来看一个以逗号分隔的`(CSV)`文本文件：

In [2]:
!type .\ex1.csv

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


由于该文件以逗号隔开,所以可用`read_csv`将其读入一个`DataFrame`:

In [3]:
df = pd.read_csv('./ex1.csv')
df

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


也可以用`read_table`，只不过需要指定分隔符而已：

In [4]:
pd.read_table('./ex1.csv', sep=',')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


并不是所有文件都有标题行：

In [5]:
!type .\ex2.csv

1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


读入该文件有两个办法。可让`pandas`为其分配默认的列名，也可自己定义列名：

In [6]:
pd.read_csv('./ex2.csv', header=None)

,0,1,2,3,4
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [7]:
pd.read_csv('./ex2.csv', names=['a', 'b', 'c', 'd', 'message'])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


若希望将`message`列做成`DataFrame`的索引。可明确表示要将该列放到索引4的位置上，也可通过`index_col`参数指定`"message"`:

In [8]:
names = ['a', 'b', 'c', 'd', 'message']
pd.read_csv('./ex2.csv', names=names, index_col='message')

,a,b,c,d
message,,,,
hello,1,2,3,4
world,5,6,7,8
foo,9,10,11,12


若希望将多个列做成一个层次化索引，只需传入由列编号或列名组成的列表即可：

In [9]:
!type .\csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [10]:
parsed = pd.read_csv('./csv_mindex.csv', index_col=['key1', 'key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

有些表格可能不是用固定的分隔符去分隔字段的（比如空白符或其他模式(此处表字符串，详情在数据结构) )。对于这种情况，可以编写一个正则表达式来作为`read_table`的分隔符。看看下面这个文本文件：

In [11]:
list(open('./ex3.txt'))

['            A         B         C\n',
 'aaa -0.264438 -1.026059 -0.619500\n',
 'bbb  0.927272  0.302904 -0.032399\n',
 'ccc -0.264273 -0.386314 -0.217601\n',
 'ddd -0.871858 -0.348382  1.100491\n']

该文件各个字段由数量不定的空白符分隔，虽然你可以对其做一些手工调整，但 这个情况还是处理比较好。本例的这个情况可以用正则表达式`\s＋`表示，于是我们就有了：

In [12]:
result = pd.read_table('./ex3.txt', sep='\s+')
result

,A,B,C
aaa,-0.264438,-1.026059,-0.619500
bbb,0.927272,0.302904,-0.032399
ccc,-0.264273,-0.386314,-0.217601
ddd,-0.871858,-0.348382,1.100491


由于列名比数据行的数量少，所以`read_table`推断第一列应该是`DataFrame`的索引。准确的说法应该是:列名的数量比列的数量少1。完整的说法应该是列名“行”中“有内容的”字段数量比其他数据“行”中“有内容的”字段数量少l

这些解析器函数还有许多参数可以帮助你处理各种各样的异形文件格式（参见表6-2)。  
![](./read_csv--read_table函数的参数.png)

比如说，你可以用`skiprows`跳过文件的第一行、第三行和第四行：

In [13]:
!type .\ex4.csv

# hey!
a,b,c,d,message
# just wanted to make things more difficult for you
# who reads CSV files with computers, anyway?
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo


In [14]:
pd.read_csv('./ex4.csv', skiprows=[0, 2, 3])

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


缺失值处理是文件解析任务中的一个重要组成部分。缺失数据经常是要么没有（空字符串），要么用某个标记值表示。默认情况下，`pandas`会用一组经常出现的标记值进行识别，如 NA、-1. #I ND以及NULL等：

In [15]:
!type .\ex5.csv

something,a,b,c,d,message
one,1,2,3,4,NA
two,5,6,,8,world
three,9,10,11,12,foo


In [16]:
result = pd.read_csv('./ex5.csv')

In [17]:
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


In [18]:
pd.isnull(result)

,something,a,b,c,d,message
0,False,False,False,False,False,True
1,False,False,False,True,False,False
2,False,False,False,False,False,False


`na_values`可接受一组用于表示缺失值的字符串：

In [19]:
result = pd.read_csv('./ex5.csv', na_values=['NULL'])
result

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


可用一个字典为各列指定不同的`NA`标记值：

In [20]:
sentinels = {'message': ['foo', 'NA'], 'something': ['two']}
pd.read_csv('./ex5.csv', na_values=sentinels)

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,NaN,5,6,NaN,8,world
2,three,9,10,11.0,12,NaN


## 逐块读取文本文件
在处理很大的文件时，或找出大文件中的参数集以便于后续处理时，你可能只想读取文件的一小部分或逐块对文件进行迭代。

In [21]:
result = pd.read_csv('./ex6.csv')
result

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q
...,...,...,...,...,...
9995,2.311896,-0.417070,-1.409599,-0.515821,L
9996,-0.479893,-0.650419,0.745152,-0.646038,E
9997,0.523331,0.787112,0.486066,1.093156,K
9998,-0.362559,0.598894,-1.843201,0.887292,G


若只想读取几行（避免读取整个文件），通过`nrows`进行指定即可：

In [22]:
pd.read_csv('./ex6.csv', nrows=5)

,one,two,three,four,key
0,0.467976,-0.038649,-0.295344,-1.824726,L
1,-0.358893,1.404453,0.704965,-0.200638,B
2,-0.501840,0.659254,-0.421691,-0.057688,G
3,0.204886,1.074134,1.388361,-0.982404,R
4,0.354628,-0.133116,0.283763,-0.837063,Q


要逐块读取文件，需要设置`chunksize`(行数)：

In [23]:
chunker = pd.read_csv('./ex6.csv', chunksize=1000)
chunker

`read_csv`所返回的这个`TextParser`对象使你可以根据`chunksize`对文件进行逐块迭代。如，可迭代处理`ex6.csv`，将值计数聚合到`“key”`列中：

In [24]:
chunker = pd.read_csv('./ex6.csv', chunksize=1000)

tot = Series([])
for piece in chunker:
    tot = tot.add(piece['key'].value_counts(), fill_value=0)

tot = tot.sort_values(ascending=False)
tot[:10]

<ipython-input-24-8a14a9fe24dd>:3: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  tot = Series([])


E    368.0
X    364.0
L    346.0
O    343.0
Q    340.0
M    338.0
J    337.0
F    335.0
K    334.0
H    330.0
dtype: float64

`TextParser`还有一个`get_chunk`方法，它使你可读取任意大小的块：

## 将数据写出到文本格式
数据也可被输出为分隔符格式文本：

In [25]:
data = pd.read_csv('./ex5.csv')
data

,something,a,b,c,d,message
0,one,1,2,3.0,4,NaN
1,two,5,6,NaN,8,world
2,three,9,10,11.0,12,foo


利用`DataFrame`的`to_csv`方法，可将数据写到一个以逗号分隔的文件中：

In [26]:
data.to_csv('./out.csv')
!type .\out.csv

,something,a,b,c,d,message
0,one,1,2,3.0,4,
1,two,5,6,,8,world
2,three,9,10,11.0,12,foo


也可使用其它分隔符（由于这里直接写出到`sys.stdout`,所以仅仅是打印出文本结果而已） ：

In [27]:
import sys
data.to_csv(sys.stdout, sep='|')

|something|a|b|c|d|message
0|one|1|2|3.0|4|
1|two|5|6||8|world
2|three|9|10|11.0|12|foo


缺失值在输出结果中会被表示为空字符串。你可能希望将其表示为别的标记值：

In [28]:
data.to_csv(sys.stdout, na_rep='NULL')

,something,a,b,c,d,message
0,one,1,2,3.0,4,NULL
1,two,5,6,NULL,8,world
2,three,9,10,11.0,12,foo


如果没有设置其他选项，则会写出行和列的标签。当然，它们也都可以被禁用：

In [29]:
data.to_csv(sys.stdout, index=False, header=False)

one,1,2,3.0,4,
two,5,6,,8,world
three,9,10,11.0,12,foo


此外，你还可以只写出一部分的列，并以你指定的顺序排列：

In [30]:
data.to_csv(sys.stdout, index=False, columns=['a', 'b', 'c'])

a,b,c
1,2,3.0
5,6,
9,10,11.0


`Series`也有一个`to_csv`方法：

In [31]:
dates = pd.date_range('1/1/2000', periods=7)
ts = Series(np.arange(7), index=dates)
ts.to_csv('./tseries.csv')
!type .\tseries.csv

,0
2000-01-01,0
2000-01-02,1
2000-01-03,2
2000-01-04,3
2000-01-05,4
2000-01-06,5
2000-01-07,6


虽然只需一点整理工作（无`header`行，第一列作索引）就能用`read_csv`读取`CSV`文件：

In [32]:
pd.read_csv('./tseries.csv', parse_dates=True)

,Unnamed: 0,0
0,2000-01-01,0
1,2000-01-02,1
2,2000-01-03,2
3,2000-01-04,3
4,2000-01-05,4
5,2000-01-06,5
6,2000-01-07,6


## 手工处理分隔符格式
大部分存储在磁盘上的表格型数据都能用`pandas.read_table`进行加载。然而，有时还是需要做一些手工处理。由干接收到含有畸形行的文件而使`read_table`出毛病的情况并不少见。为了说明这些基本工具，看看下面这个简单的`CSV`文件：

In [33]:
!type .\ex7.csv

"a","b","c"
"1","2","3"
"1","2","3"


对干任何单字符分隔符文件，可以直接使用`Python`内置的`CSV`模块。将任意已打开的文件或文件型的对象传给`csv.reader`:

In [34]:
import csv
f = open('./ex7.csv')
reader = csv.reader(f)

对这个`reader`进行迭代将会为每行产生一个元组(这里得到的结果不是元组而是列表)(并移除了所有的引号)：

In [35]:
for line in reader:
    print(line)

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3']


为了使数据合乎要求，需要对其做一些整理工作：

In [36]:
lines = list(csv.reader(open('./ex7.csv')))
header, values = lines[0], lines[1:]
data_dict = {h: v for h, v in zip(header, zip(*values))}
data_dict

{'a': ('1', '1'), 'b': ('2', '2'), 'c': ('3', '3')}

`csv`文件的形式有很多。只需定义`csv.Dialect`的一个子类即可定义出新格式(如专门的分隔符、字符串引用约定、行结束符等):

In [37]:
class my_dialect(csv.Dialect):
    lineterminator = '\n'
    delimiter = '；'
    quotechar = '"'
    quoting = 0
reader = csv.reader(f, dialect=my_dialect)

各个`csv`语支的参数也可用关键字的形式提供给`csv.reader`，而无需定义子类：

In [38]:
reader = csv.reader(f, delimiter='|')

![](./csv语支选项.png)  
注：对于那些使用复杂分隔符或多字符分隔符的文件，`csv`模块就无能为力了。这种情况下，你就只能使用字符串的`split`方法或正则表达式方法`re.split`进行行拆分和其他整理工作了。

要手工输出分隔符文件，你可以使用`csv.writer`。它接受一个已打开且可写的文件对象以及跟`csv.reader`相同的那些语支和格式化选项 ：

In [39]:
with open('mydata.csv', 'w') as f:
    writer = csv.writer(f, dialect=my_dialect)
    writer.writerow(('one', 'two', 'three'))
    writer.writerow(('1', '2', '3'))
    writer.writerow(('4', '5', '6'))
    writer.writerow(('7', '8', '9'))

## JSON数据
JSON (JavaScript Object Notation的简称）已经成为通过`HTTP`请求在`Web`浏览器和其他应用程序之间发送数据的标准格式之一。它是一种比表格型文本格式（如`CSV`)灵活得多的数据格式。下面是一个例子：

In [40]:
obj = """
{"name": "Wes",
 "places_lived": ["United States", "Spain", "Germany"],
 "pet": null,
 "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]},
              {"name": "Katie", "age": 38,
               "pets": ["Sixes", "Stache", "Cisco"]}]
}
"""

除其空值`null`和一些其他的细微差别（如列表末尾不允许存在多余的逗号）之外,`JSON`非常接近于有效的`Python`代码。基本类型有对象（字典）、数组（列表）、字符串、数值、布尔值以及`null`。对象中所有的键都必须是字符串。许多`Python`库都可以读写`JSON`数据。我将使用`json`,因为它是构建于`Python`标准库中的。通过`json.loads`即可将`JSON` 字符串转换成`Python`形式：

In [41]:
import json
result = json.loads(obj)
result

{'name': 'Wes',
 'places_lived': ['United States', 'Spain', 'Germany'],
 'pet': None,
 'siblings': [{'name': 'Scott', 'age': 30, 'pets': ['Zeus', 'Zuko']},
  {'name': 'Katie', 'age': 38, 'pets': ['Sixes', 'Stache', 'Cisco']}]}

`json.dumps`则将`Python`对象转换成`JSON`格式：

In [42]:
asjson = json.dumps(result)
asjson

'{"name": "Wes", "places_lived": ["United States", "Spain", "Germany"], "pet": null, "siblings": [{"name": "Scott", "age": 30, "pets": ["Zeus", "Zuko"]}, {"name": "Katie", "age": 38, "pets": ["Sixes", "Stache", "Cisco"]}]}'

如何将(一个或一组)`JSON`对象转换为`DataFrame`或其他便于分析的数据结构就由你决定了。最简单方便的方式是：向`DataFrame`构造器传入一组`JSON`对象，并选取数据字段的子集:

In [43]:
siblings = DataFrame(result['siblings'], columns=['name', 'age'])
siblings

,name,age
0,Scott,30
1,Katie,38


## XML和HTML: Web信息收集
`Python`有许多可以读写`HTML`和`XML`格式数据的库。`lxml`(http://lxml.de) 就是其中之一，它能够高效且可靠地解析大文件。`lxml`有多个编程接口。首先我要用`lxml.html`处理`HTML`,然后再用`lxml.objectify`做一些`XML`处理。  
许多网站都将数据放到`HTML`表格中以便在浏览器中查看，但不能以一种更易于机器阅读的格式（如`JSON、HTML`或`XML`) 进行下载。我发现`Ya hoo! Finance`的股票期权数据就是这样。可能你对这种数据不熟悉：期权是指使你有权从现在开始到未来某个时间（到期日）内以某个特定价格（执行价）买进（看涨期权）或卖出（乔跌期权）某公司股票的衍生合约。人们的看涨和看跌期权交易有多种执行价和到期日，这些数据都可以在`Yahoo! Finance`的各种表格中找到。  
首先，找到你希望获取数据的`URL`,利用`urllib2`将其打开，然后用`lxml`解析得到的数据流，如下所示：

In [44]:
from lxml.html import parse
from urllib import request
parsed = parse(request.urlopen('http://finance.yahoo.com/q/op?s=AAPL+Options'))
doc = parsed.getroot()

通过这个对象，你可以获取特定类型的所有`HTML`标签 (`tag`)，比如含有所需数据的`table`标签。给这个简单的例子加点启发性，假设你想得到该文档中所有的`URL`链接。`HTML`中的链接是`a`标签。使用文档根节点的`findall`方法以及一个`XPath`（对文档的“查询＂的一种表示手段）：

In [45]:
links = doc.findall('.//a')
links[15:20]

[<Element a at 0x21373995680>,
 <Element a at 0x21373995f90>,
 <Element a at 0x213739957c0>,
 <Element a at 0x21373995770>,
 <Element a at 0x21373995720>]

但这些是表示`HTML`元素的对象。要得到`URL`和链接文本，你必须使用各对象的`get`方法（针对`URL`)和`text _content`方法（针对显示文本）：

In [46]:
lnk = links[10]
lnk

<Element a at 0x21373995180>

In [47]:
lnk.get('href')

'https://login.yahoo.com/config/login?.src=finance&.intl=us&.lang=en-US&.done=https%3A%2F%2Ffinance.yahoo.com%2Fquote%2FAAPL%2Foptions%3Fltr%3D1&activity=uh-signin&pspid=28951412'

In [48]:
lnk.text_content()

'Sign in'

因此，编写下面这条列表推导式即可获取文档中的全部`URL`:

In [49]:
urls = [lnk.get('href') for lnk in doc.findall('.//a')]
urls[-10:]

['https://help.yahoo.com/kb/finance-for-web/SLN2310.html?locale=en_US',
 'https://help.yahoo.com/kb/finance-for-web',
 'https://yahoo.uservoice.com/forums/382977',
 'https://policies.oath.com/us/en/oath/privacy/index.html',
 'https://policies.oath.com/us/en/oath/privacy/adinfo/index.html',
 'https://www.verizonmedia.com/policies/us/en/verizonmedia/terms/otos/index.html',
 'https://finance.yahoo.com/sitemap/',
 'https://twitter.com/YahooFinance',
 'https://facebook.com/yahoofinance',
 'https://www.linkedin.com/company/yahoo-finance']

现在，从文档中找出正确表格的办法就是反复试验了。有些网站会给目标表格加上一个`id`属性。我确定有两个分别放置看涨数据和看跌数据的表格：

In [50]:
tables = doc.findall('.//table')
calls = tables[0]
puts = tables[1]

每个表格都有一个标题行，然后才是数据行：

In [51]:
rows = calls.findall('.//tr')

对于标题行和数据行，我们希望获取每个单元格内的文本。对于标题行，就是`th`单元格，而对于数据行，则是`td`单元格：

In [52]:
def _unpack(row, kind='td'):
    elts = row.findall('.//%s' % kind)
    return [val.text_content() for val in elts]

这样，就得到了：

In [53]:
_unpack(rows[0], kind='th')

['Contract Name',
 'Last Trade Date',
 'Strike',
 'Last Price',
 'Bid',
 'Ask',
 'Change',
 '% Change',
 'Volume',
 'Open Interest',
 'Implied Volatility']

In [54]:
_unpack(rows[1], kind='td')

['AAPL210122C00065000',
 '2021-01-19 11:51AM EST',
 '65.00',
 '62.50',
 '62.45',
 '63.20',
 '+0.10',
 '+0.16%',
 '43',
 '86',
 '395.51%']

现在，把所有步骤结合起来，将数据转换为一个`DataFrame`。由于数值型数据仍然是字符串格式，所以我们希望将部分列（可能不是全部）转换为浮点数格式。虽然你可以手工实现该功能，但是`pandas`恰好就有一个`TextParser`类可用于自动类型转换(`read_csv`和其他解析函数其实在内部都用到了它）：

In [55]:
from pandas.io.parsers import TextParser
def parse_options_data(table):
    rows = table.findall('.//tr')
    header = _unpack(rows[0], kind='th')
    data = [_unpack(r) for r in rows[1:]]
    return TextParser(data, names=header).get_chunk()

最后，对那两个`lxml`表格对象调用该解析函数并得到最终的`DataFrame`: 

In [56]:
call_data = parse_options_data(calls)
put_data = parse_options_data(puts)
call_data[:10]

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask,Change,% Change,Volume,Open Interest,Implied Volatility
0,AAPL210122C00065000,2021-01-19 11:51AM EST,65.0,62.50,62.45,63.20,0.10,+0.16%,43,86,395.51%
1,AAPL210122C00070000,2021-01-19 9:43AM EST,70.0,58.05,57.45,58.15,0.79,+1.38%,22,84,348.05%
2,AAPL210122C00075000,2021-01-15 3:43PM EST,75.0,52.25,52.45,53.15,0.00,-,49,266,312.70%
3,AAPL210122C00080000,2021-01-19 12:12AM EST,80.0,49.17,47.45,48.15,0.00,-,25,400,279.30%
4,AAPL210122C00085000,2021-01-19 9:37AM EST,85.0,42.45,42.45,43.15,0.20,+0.47%,223,206,247.85%
5,AAPL210122C00090000,2021-01-19 2:34PM EST,90.0,37.85,37.45,38.20,0.57,+1.53%,400,772,223.83%
6,AAPL210122C00094000,2021-01-19 2:34PM EST,94.0,33.85,33.45,34.20,-1.10,-3.15%,60,-,200.39%
7,AAPL210122C00095000,2021-01-19 9:37AM EST,95.0,33.50,32.70,33.20,-0.02,-0.06%,40,42,160.16%
8,AAPL210122C00096000,2021-01-19 9:53AM EST,96.0,31.55,31.40,32.20,-1.60,-4.83%,24,-,188.87%
9,AAPL210122C00099000,2021-01-19 12:12AM EST,99.0,28.20,28.40,29.20,-3.80,-11.87%,20,-,171.88%


## 利用lxml.objectify解析XML
`XML(Extensible Markup Language)`是另一种常见的支持分层、嵌套数据以及元数据的结构化数据格式。本书所使用的这些文件实际上来自于一个很大的`XML`文档。  
之前，我介绍了`lxml`库及其`lxml.html`接口。这里我将介绍另一个用于操作`XML`数据的接口，即`lxml.objectify`。  
纽约大都会运输署(Metropolitan Transportation Authority, MTA) 发布了一些有关其公交和列车服务的数据资料 ( http://www.mta.info/developers/download.html )。这里,我们将看看包含在一组`XML`文件中的运行情况数据。每项列车或公交服务都有各自的文件
（如`Metro-NorthRailroad`的文件是`Performance_MNR.xml`)，其中每条`XML`记录就是一条月度数据，如下所示 ：  

```<INDICATOR>
  <INDICATOR_SEQ>373889</INDICATOR_SEQ>
  <PARENT_SEQ></PARENT_SEQ>
  <AGENCY_NAME>Metro-North Railroad</AGENCY_NAME>
  <INDICATOR_NAME>Escalator Availability</INDICATOR_NAME>
  <DESCRIPTION>Percent of the time that escalators are operational
  systemwide. The availability rate is based on physical observations performed
  the morning of regular business days only. This is a new indicator the agency
  began reporting in 2009.</DESCRIPTION>
  <PERIOD_YEAR>2011</PERIOD_YEAR>
  <PERIOD_MONTH>12</PERIOD_MONTH>
  <CATEGORY>Service Indicators</CATEGORY>
  <FREQUENCY>M</FREQUENCY>
  <DESIRED_CHANGE>U</DESIRED_CHANGE>
  <INDICATOR_UNIT>%</INDICATOR_UNIT>
  <DECIMAL_PLACES>1</DECIMAL_PLACES>
  <YTD_TARGET>97.00</YTD_TARGET>
  <YTD_ACTUAL></YTD_ACTUAL>
  <MONTHLY_TARGET>97.00</MONTHLY_TARGET>
  <MONTHLY_ACTUAL></MONTHLY_ACTUAL>
</INDICATOR>```

先用`lxml.objectify`解析该文件，然后通过`getroot`得到该`XML`文件的根节点的引用：

In [57]:
from lxml import objectify
path = './Performance_MNR.xml'
parsed = objectify.parse(open(path))
root = parsed.getroot()

`root.INDICATOR`返回一个用于产生各个`<INDICATOR> XML`元素的生成器。对于每条记录,我们可以用标记名（如`YTD_ ACTUAL`)和数据值填充一个字典（排除几个标记）:

In [58]:
data = []

skip_fields = ['PARENT_SEQ', 'INDICATOR_SEQ',
               'DESIRED_CHANGE', 'DECIMAL_PLACES']

for elt in root.INDICATOR:
    el_data = {}
    for child in elt.getchildren():
        if child.tag in skip_fields:
            continue
        el_data[child.tag] = child.pyval
    data.append(el_data)

最后将这组字典转换为一个`DataFrame`:

In [59]:
perf = DataFrame(data)
perf

,AGENCY_NAME,INDICATOR_NAME,DESCRIPTION,PERIOD_YEAR,PERIOD_MONTH,CATEGORY,FREQUENCY,INDICATOR_UNIT,YTD_TARGET,YTD_ACTUAL,MONTHLY_TARGET,MONTHLY_ACTUAL
0,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,1,Service Indicators,M,%,95,96.9,95,96.9
1,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,2,Service Indicators,M,%,95,96,95,95
2,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,3,Service Indicators,M,%,95,96.3,95,96.9
3,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,4,Service Indicators,M,%,95,96.8,95,98.3
4,Metro-North Railroad,On-Time Performance (West of Hudson),Percent of commuter trains that arrive at thei...,2008,5,Service Indicators,M,%,95,96.6,95,95.8
...,...,...,...,...,...,...,...,...,...,...,...,...
643,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,8,Service Indicators,M,%,97,,97,
644,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,9,Service Indicators,M,%,97,,97,
645,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,10,Service Indicators,M,%,97,,97,
646,Metro-North Railroad,Escalator Availability,Percent of the time that escalators are operat...,2011,11,Service Indicators,M,%,97,,97,


`XML`数据可以比本例复杂得多。每个标记都可以有元数据。看看下面这个`HTML`的链接标记（它也贷是一段有效的`XML)	:

In [60]:
from io import StringIO
tag = '<a href="http://www.google.com">Google</a>'
root = objectify.parse(StringIO(tag)).getroot()

现在就可以访问链接文本或标记中的任何字段了(如`href`):

In [61]:
root

<Element a at 0x2137473ef00>

In [62]:
root.get('href')

'http://www.google.com'

In [63]:
root.text

'Google'

## 二进制数据格式
实现数据的二进制格式存储最简单的办法之一是使用`Python`内置的`pickle`序列化。为了使用方便，`pandas`对象都有一个用于将数据以`pickle`形式保存到磁盘上的`to_pickle`方法:

In [64]:
frame = pd.read_csv('./ex1.csv')
frame

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


In [66]:
frame.to_pickle('./frame_pickle')

可通过另一个也很好用的`pickle`函数`pandas.read_pickle`将数据读回到`Python`:

In [68]:
pd.read_pickle('./frame_pickle')

,a,b,c,d,message
0,1,2,3,4,hello
1,5,6,7,8,world
2,9,10,11,12,foo


警告：`pickle`仅建议用干短期存储格式。其原因是很难保证该格式永远是稳定的；今天`pickle`的对象可能无法被后续版本的库`unpickle`出来。虽然我尽力保证这种事情不会发生在`pandas`中，但是今后的某个时候说不定还是得“打破＂该`pickle`格式。

## 使用HDF5格式
很多工具都能实现高效读写磁盘上以二进制格式存储的科学数据。`HDF5`就是其中一个流行的工业级库，它是一个C库，带有许多语言的接口，如`Java 、Python`和`MATLAB`等。`HDF5`中的`HDF`指的是层次型数据格式(hierarchical data format)。每个`HDF5`文件都含有一个文件系统式的节点结构，它使你能够存储多个数据集并支持元数据。与其他简单格式相比，`HDF5`支持多种压缩器的即时压缩，还能更高效地存储重复模式数据。对于那些非常大的无法直接放入内存的数据集，`HDF5`就是不错的选择，因为它可以高效地分块读写。  
`Python`中的`HDF5`库有两个接口（即`PyTables`和`h5py`) ，它们各自采取了不同的问题解决方式。`h5py`提供了一种直接而高级的`HDF5 API`访问接口，而`PyTables`则抽象了`HDF5`的许多细节以提供多种灵活的数据容器、表索引、查询功能以及对核外计算技术(out-of-core computation)的某些支持。  
`pandas`有一个最小化的类似干字典的`HDFStore`类，它通过`PyTables`存储`pandas`对象：

In [72]:
frame = pd.DataFrame({'a': np.random.randn(100)})
store = pd.HDFStore('mydata.h5')
store['obj1'] = frame
store['obj1_col'] = frame['a']
store

<class 'pandas.io.pytables.HDFStore'>
File path: mydata.h5

`HDF5`文件中的对象可通过与字典一样的方式进行获取：

In [73]:
store['obj1']

,a
0,0.507781
1,0.990332
2,0.849054
3,-1.446520
4,-0.992702
...,...
95,1.138126
96,-1.048164
97,-1.332782
98,0.378811


如果需要处理海最数据，我建议你好好研究一下`PyTables`和`h5py`,看看它们能满足你的哪些需求。由于许多数据分析问题都是`IO`密集型（而不是`CPU`密集型），利用`HDF5`这样的工具能显著提升应用程序的效率。  
警告：`HDF5`不是数据库。它最适合用作“一次写多次读＂的数据集。虽然数据可以在任何时候被添加到文件中，但如果同时发生多个写操作，文件就可能会被破坏。

## 读取Microsoft Excel文件
`pandas`的`ExcelFile`类或`pandas.read_excel`函数⽀持读取存储在Excel 2003（或更⾼版本）中的表格型数据。这两个⼯具分别使⽤扩展包`xlrd`和`openpyxl`读取`XLS`和`XLSX`⽂件。你可以⽤`pip`或`conda`安装它们。
要使⽤`ExcelFile`，通过传递`xls`或`xlsx`路径创建⼀个实例：

In [76]:
xlsx = pd.ExcelFile('./ex1.xlsx')

存储在表单中的数据可以`read_excel`读取到`DataFrame`:

In [77]:
pd.read_excel(xlsx, 'Sheet1')

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果要读取⼀个⽂件中的多个表单，创建`ExcelFile`会更快，但你也可以将⽂件名传递到`pandas.read_excel`：

In [78]:
frame = pd.read_excel('./ex1.xlsx', 'Sheet1')
frame

,Unnamed: 0,a,b,c,d,message
0,0,1,2,3,4,hello
1,1,5,6,7,8,world
2,2,9,10,11,12,foo


如果要将`pandas`数据写⼊为`Excel`格式，你必须⾸先创建⼀个`ExcelWriter`，然后使⽤`pandas`对象的`to_excel`⽅法将数据写⼊到其中：

In [80]:
writer = pd.ExcelWriter('./ex2.xlsx')
frame.to_excel(writer, 'Sheet1')
writer.save()

还可以不使用`ExcelWriter`,而是传递文件的路径到`to_excel`:

In [85]:
frame.to_excel('./ex2.xlsx')


## Web APIs交互
许多⽹站都有⼀些通过JSON或其他格式提供数据的公共`API`。通过`Python`访问这些`API`的办法有不少。⼀个简单易⽤的办法（推荐）是`requests`包（http://docs.python-requests.org ）。  
为了搜索最新的30个`GitHub`上的`pandas`主题，我们可以发⼀个`HTTP GET`请求，使⽤`requests`扩展库：

In [86]:
import requests
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
resp = requests.get(url)
resp

<Response [200]>

响应对象的`json`⽅法会返回⼀个包含被解析过的`JSON`字典，加载到⼀个`Python`对象中：

In [87]:
data = resp.json()
data[0]['title']

'TST: test_to_csv_compression_encoding_gcs fails sometimes'

`data`中的每个元素都是⼀个包含所有`GitHub`主题⻚数据（不包含评论）的字典。我们可以直接传递数据到`DataFrame`，并提取感兴趣的字段：

In [88]:
issues = pd.DataFrame(data, columns=['number', 'title',
                                     'labels', 'state'])
issues

,number,title,labels,state
0,39286,TST: test_to_csv_compression_encoding_gcs fail...,"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
1,39285,pandas-dev issue #21881,[],open
2,39283,CLN: Remove np_datetime64_compat and np_array_...,"[{'id': 211029535, 'node_id': 'MDU6TGFiZWwyMTE...",open
3,39282,Replaced empty series check with Series with c...,[],open
4,39280,BUG: DataFrame.__setitem__ raising ValueError ...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
5,39279,REF: share start_time/end_time between Period/...,[],open
6,39278,BUG: loc.setitem raising ValueError when df ha...,"[{'id': 2822098, 'node_id': 'MDU6TGFiZWwyODIyM...",open
7,39277,REF: share PeriodArray.asfreq with Period.asfreq,"[{'id': 60635328, 'node_id': 'MDU6TGFiZWw2MDYz...",open
8,39276,BUG: is_utc(dateutil_utc),"[{'id': 76811, 'node_id': 'MDU6TGFiZWw3NjgxMQ=...",open
9,39275,ENH: apply np.ufunc.accumulate along the colum...,"[{'id': 76812, 'node_id': 'MDU6TGFiZWw3NjgxMg=...",open


## 数据库交互
在商业场景下，⼤多数数据可能不是存储在⽂本或`Excel`⽂件中。基于`SQL`的关系型数据库（如`SQL Server、PostgreSQL`和`MySQL`等）使⽤⾮常⼴泛，其它⼀些数据库也很流⾏。数据库的选择通常取决于性能、数据完整性以及应⽤程序的伸缩性需求。  
将数据从`SQL`加载到`DataFrame`的过程很简单，此外`pandas`还有⼀些能够简化该过程的函数。例如，我将使⽤`SQLite`数据库（通过Python内置的sqlite3驱动器）：

In [91]:
import sqlite3
query = """
CREATE TABLE test
(a VARCHAR(20), b VARCHAR(20),
 c REAL,        d INTEGER
);"""
con = sqlite3.connect('mydata.sqlite')
con.execute(query)

插入数据：

In [92]:
data = [('Atlanta', 'Georgia', 1.25, 6),
        ('Tallahassee', 'Florida', 2.6, 3),
        ('Sacramento', 'California', 1.7, 5)]
stmt = "INSERT INTO test VALUES(?, ?, ?, ?)"
con.executemany(stmt, data)
con.commit()

从表中选取数据时，⼤部分`Python SQL`驱动器（`PyODBC、psycopg2、MySQLdb、pymssql`等）都会返回⼀个元组列表：

In [94]:
cursor = con.execute('select * from test')
rows = cursor.fetchall()
rows

[('Atlanta', 'Georgia', 1.25, 6),
 ('Tallahassee', 'Florida', 2.6, 3),
 ('Sacramento', 'California', 1.7, 5)]

你可以将这个元组列表传给`DataFrame`构造器，但还需要列名（位于光标的`description`属性中）：

In [95]:
cursor.description

(('a', None, None, None, None, None, None),
 ('b', None, None, None, None, None, None),
 ('c', None, None, None, None, None, None),
 ('d', None, None, None, None, None, None))

In [96]:
pd.DataFrame(rows, columns=[x[0] for x in cursor.description])

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5


这种数据规整操作相当多，你肯定不想每查⼀次数据库就重写⼀次。`SQLAlchemy`项⽬是⼀个流⾏的`Python SQL`⼯具，它抽象出了`SQL`数据库中的许多常⻅差异。`pandas`有⼀个`read_sql`函数，可以让你轻松的从`SQLAlchemy`连接读取数据。这⾥，我们⽤`SQLAlchemy`连接`SQLite`数据库，并从之前创建的表读取数据：

In [97]:
import sqlalchemy as sqla
db = sqla.create_engine('sqlite:///mydata.sqlite')
pd.read_sql('select * from test', db)

,a,b,c,d
0,Atlanta,Georgia,1.25,6
1,Tallahassee,Florida,2.60,3
2,Sacramento,California,1.70,5
